# 클라이언트

In [ ]:
import socket
import firebase_admin
import os
import time
from uuid import uuid4
from firebase_admin import credentials
from firebase_admin import storage


# 파이어베이스 업로드 부분
def fileUpload(bucket, cs, path, name):
    blob = bucket.blob('스토리지 내 디렉터리명'+ name)
    # 액세스 토큰 설정
    new_token = uuid4()
    metadata = {"firebaseStorageDownloadTokens": new_token}
    blob.metadata = metadata
 
    # upload file
    blob.upload_from_filename(filename = path + "/" + name, content_type='image/jpg')
    message = name + "=" + str(new_token)
    cs.send(message.encode())
    print(name, " uploaded")
    time.sleep(2) # 속도가 달라 메세지가 겹치는 현상 방지


# 디렉터리에서 파일을 불러와 업로드 함수 호출
def loadDir(bucket, cs):
    path = '업로드할 이미지가 있는 디렉터리'
    img_list = os.listdir(path)
    for name in img_list:
        fileUpload(bucket, cs, path, name)



# 클라이언트 통신 시작 부분
def client():
    # 파이어베이스
    cred = credentials.Certificate("키가 있는 경로")
    default_app = firebase_admin.initialize_app(cred,{'storageBucket': "스토리지 이름"})
    bucket = storage.bucket()

    # 소켓 통신 연결
    HOST = '서버 ip 주소'
    PORT = 9999
    cs = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    cs.connect((HOST, PORT))

    # 업로드 및 통신 시작
    loadDir(bucket, cs)  
    
    # 통신 종료
    message = "finish"
    cs.send(message.encode())
    cs.close()
    

client()
print("연결 종료")

# 서버

In [ ]:
import socket
from requests import get

def parsing(fullpath):
    idx1 = fullpath.find("image")
    idx2 = fullpath.find("=")
    name = fullpath[idx1 : idx2]
    token = fullpath[idx2 + 1:]
    return name, token


def download(fileName, token):
    url = "이미지 url" + fileName + "?alt=media&token=" + token
    with open(fileName, "wb") as file:   
        response = get(url)             
        file.write(response.content)    


# 클라이언트에게 메세지를 받고 파싱해서 이미지를 다운로드
def receive(cs):

    print('client socket connected')
    err = 1

    while True:
        try:
            data = cs.recv(1024)

            if not data:
                print("client socket disconnected")
                break

            if str(data.decode()) == "finish":
                err = 0
                break

            else:
                name, token = parsing(str(data.decode()))
                print(name, token)
                download(name, token)

        except ConnectionResetError as e:
            print("client socket disconnected")
            print(f"e: {e}")
            break

    cs.close()
    print("client socket disconnected")
    # 정상 종료 되었을 경우, 뒤의 로직을 수행
    if err == 0:
        next()


# 다음에 실행되어야 하는 함수
def next():
    print('다음 함수 실행')
            

# 서버 측 함수
def server():
    ip = '서버 ip 주소'
    port = 9999 

    # 소켓 초기화
    ss = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

    # 소켓 에러처리
    ss.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)

    ss.bind((ip, port))

    ss.listen()

    print('server start')

    while(True):
        cs, addr = ss.accept()
        receive(cs)

        
server()        